In [1]:
import torch  # missing from lecture 2
import torch.nn as nn

## Create input_tensor with three features
input_tensor = torch.tensor([[0.3471, 0.4547, -0.2356]])
print(input_tensor.shape)
input_tensor

torch.Size([1, 3])


tensor([[ 0.3471,  0.4547, -0.2356]])

In [2]:
# define a linear layer
linear_layer = nn.Linear(in_features=3, out_features=2)
output = linear_layer(input_tensor)
print(output)

tensor([[ 0.6837, -0.3210]], grad_fn=<AddmmBackward0>)


In [3]:
print(linear_layer.weight.shape)
linear_layer.weight
# Parameter containing:
# tensor([[ 0.1666,  0.4067, -0.4190],
#         [-0.1845,  0.0945, -0.2244]], requires_grad=True)

torch.Size([2, 3])


Parameter containing:
tensor([[-0.1789,  0.4501, -0.4519],
        [-0.2985, -0.4944,  0.2654]], requires_grad=True)

In [4]:
linear_layer.bias
# Parameter containing:
# tensor([-0.2581, -0.1212], requires_grad=True)

Parameter containing:
tensor([0.4347, 0.0700], requires_grad=True)

In [5]:
# ex 2
# Create a neural network of linear layers that takes a tensor of dimensions 1x8 as input and outputs a tensor of dimensions 1x1
# Use any output dimension for the first layer you want.
input_tensor = torch.Tensor([[2, 3, 6, 7, 9, 3, 2, 1]])

# Implement a small neural network with exactly two linear layers
model = nn.Sequential(nn.Linear(8, 3),  # using 3 as the output dim of the first layer, but could be anything
                      nn.Linear(3, 1)
                     )

output = model(input_tensor)
print(output)  # 1x1 output

tensor([[-1.4369]], grad_fn=<AddmmBackward0>)


In [6]:
# sigmoid activation
input_tensor = torch.tensor([[6.0]])
sigmoid = nn.Sigmoid()
output = sigmoid(input_tensor)
output

tensor([[0.9975]])

In [7]:
# softmax for multi-dim classification
# Create an input tensor
input_tensor = torch.tensor([[4.3, 6.1, 2.3]])
# Apply softmax along the last dimension
probabilities = nn.Softmax(dim=-1)
output_tensor = probabilities(input_tensor)
print(output_tensor)

tensor([[0.1392, 0.8420, 0.0188]])


In [8]:
# Binary classification: forward pass
# Create input data of shape 5x6
input_data = torch.tensor(
[[-0.4421, 1.5207, 2.0607, -0.3647, 0.4691, 0.0946],
[-0.9155, -0.0475, -1.3645, 0.6336, -1.9520, -0.3398],
[ 0.7406, 1.6763, -0.8511, 0.2432, 0.1123, -0.0633],
[-1.6630, -0.0718, -0.1285, 0.5396, -0.0288, -0.8622],
[-0.7413, 1.7920, -0.0883, -0.6685, 0.4745, -0.4245]])

# Create binary classification model
model = nn.Sequential(
nn.Linear(6, 4), # First linear layer
nn.Linear(4, 1), # Second linear layer
nn.Sigmoid() # Sigmoid activation function
)
# Pass input data through model
output = model(input_data)
print(output)

tensor([[0.6702],
        [0.4348],
        [0.5445],
        [0.4950],
        [0.5525]], grad_fn=<SigmoidBackward0>)


In [9]:
# lecture 4 excercises
input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

# Implement a small neural network for binary classification
model = nn.Sequential(
  nn.Linear(8, 1),
  nn.Sigmoid()
)

output = model(input_tensor)
print(output)

tensor([[0.0019]], grad_fn=<SigmoidBackward0>)


In [10]:
# Create a neural network with exactly four linear layers, which takes
# the input tensor as input, and outputs a regression value, using any
# shapes you like for the hidden layers.

input_tensor = torch.Tensor([[3, 4, 6, 7, 10, 12, 2, 3, 6, 8, 9]])

# Implement a neural network with exactly four linear layers
model = nn.Sequential(
    nn.Linear(11, 7),
    nn.Linear(7, 6),
    nn.Linear(6, 5),
    nn.Linear(5, 1)
)

output = model(input_tensor)
print(output)

tensor([[-0.7475]], grad_fn=<AddmmBackward0>)


In [11]:
# A similar neural network to the one you just built is provided, containing four linear layers;
# update this network to perform a multi-class classification with four outputs.
input_tensor = torch.Tensor([[3, 4, 6, 7, 10, 12, 2, 3, 6, 8, 9]])

# Update network below to perform a multi-class classification with four labels
model = nn.Sequential(
  nn.Linear(11, 20),
  nn.Linear(20, 12),
  nn.Linear(12, 6),
  nn.Linear(6, 4), 
  nn.Softmax(dim=-1)
)

output = model(input_tensor)
print(output)

tensor([[0.0636, 0.2537, 0.4786, 0.2041]], grad_fn=<SoftmaxBackward0>)


In [12]:
# lecture 5

In [13]:
import torch.nn.functional as F
# one-hot encoding
print(F.one_hot(torch.tensor(0), num_classes = 3))
print(F.one_hot(torch.tensor(1), num_classes = 3))
print(F.one_hot(torch.tensor(2), num_classes = 3))

tensor([1, 0, 0])
tensor([0, 1, 0])
tensor([0, 0, 1])


In [14]:
from torch.nn import CrossEntropyLoss

scores = torch.tensor([[-0.1211, 0.1059]])
one_hot_target = torch.tensor([[1, 0]])
criterion = CrossEntropyLoss()
criterion(scores.double(), one_hot_target.double())

tensor(0.8131, dtype=torch.float64)

In [15]:
# try it manually, start with softmax probabilities
import numpy as np
scores = [-0.1211, 0.1059]
targets = [1, 0]
denom = np.exp(scores[0]) + np.exp(scores[1])
p0 = np.exp(scores[0]) / denom
p1 = np.exp(scores[1]) / denom
print(p0, p1)

0.443492440036494 0.5565075599635061


In [16]:
# simplified for single point, binary classification
# for the general multi-nomial version see: https://rpubs.com/mszczepaniak/classificationgoodness
cross_entropy = -(np.log(p0))
print(cross_entropy)  # very close: 0.81307

0.8130745235187279


In [17]:
# lecture 5 excercises
y = 1
num_classes = 3

# Create the one-hot encoded vector using NumPy
one_hot_numpy = np.array([0, 1, 0])

# Create the one-hot encoded vector using PyTorch
one_hot_pytorch = F.one_hot(torch.tensor(1), num_classes)
print(one_hot_numpy)
print(one_hot_pytorch)

[0 1 0]
tensor([0, 1, 0])


Start by creating a one-hot encoded vector of the ground truth label y, which is a required step to compare y with the scores predicted by your model. Next, you'll create a cross entropy loss function. Last, you'll call the loss function, which takes scores (model predictions before the final softmax function), and the one-hot encoded ground truth label, as inputs. It outputs a single float, the loss of that sample.

In [18]:
y = [2]
scores = torch.tensor([[0.1, 6.0, -2.0, 3.2]])

# Create a one-hot encoded vector of the label y
one_hot_label = F.one_hot(torch.tensor(y), num_classes = scores.shape[1])
print(one_hot_label)
# Create the cross entropy loss function
criterion = nn.CrossEntropyLoss()
# Calculate the cross entropy loss
loss = criterion(scores.double(), one_hot_label.double())
print(loss)

tensor([[0, 0, 1, 0]])
tensor(8.0619, dtype=torch.float64)


In [19]:
# lecture 6 - Backpropagation in PyTorch
# exercise 1
weight = torch.tensor([[-1.5349,  1.1395, -0.7303, -1.4280,  1.4367,  1.5740,  0.8475,  1.3379, 1.5674],
                       [ 1.1543,  1.2855,  0.5122,  0.4215,  0.4982,  1.5640, -0.9705, -0.4987, 0.6331]])
bias = torch.tensor([-0.1435,  1.6498])
preds = torch.tensor([[2.1289, 3.7059]])
target = torch.tensor([[1., 0.]])

In [20]:
criterion = nn.CrossEntropyLoss()

# Calculate the loss
loss = criterion(preds, target)

# Compute the gradients of the loss
# torch.set_grad_enabled(True)  # Context-manager 
loss.requires_grad = True
# loss.retain_grad()
loss.backward()

# Display gradients of the weight and bias tensors in order
print(weight.grad)
print(bias.grad)

None
None


This is the output I get in the DataCamp environment:

`tensor([[-0.5063, -0.4353, -0.3859, -0.3938, -0.5257, -0.1628, -0.5167, -0.4315,
         -0.6264],
        [ 0.5063,  0.4353,  0.3859,  0.3938,  0.5257,  0.1628,  0.5167,  0.4315,
          0.6264]])
tensor([-0.8288,  0.8288])

<script.py> output:
    tensor([[-0.5215, -0.4484, -0.3975, -0.4056, -0.5416, -0.1677, -0.5322, -0.4444,
             -0.6453],
            [ 0.5215,  0.4484,  0.3975,  0.4056,  0.5416,  0.1677,  0.5322,  0.4444,
              0.6453]])
    tensor([-0.8537,  0.8537])`
`

In [21]:
model = nn.Sequential(nn.Linear(16, 8),
                      nn.Sigmoid(),
                      nn.Linear(8, 2))

# Access the weight of the first linear layer
weight_0 = model[0].weight

# Access the bias of the second linear layer
bias_1 = model[2].bias

In [22]:
model = nn.Sequential(
    nn.Linear(in_features=16, out_features=8, bias=True),
    nn.Linear(in_features=8, out_features=4, bias=True),
    nn.Linear(in_features=4, out_features=2, bias=True)
)

weight0 = model[0].weight
weight1 = model[1].weight
weight2 = model[2].weight

print(weight0)

Parameter containing:
tensor([[-0.1474, -0.0907,  0.0355, -0.0034, -0.2320, -0.0343, -0.0239,  0.1238,
          0.0715,  0.2001,  0.1781,  0.1280, -0.0649,  0.0769,  0.0991,  0.0659],
        [ 0.0124, -0.1377,  0.0165, -0.0508,  0.1346, -0.0372, -0.2265, -0.0657,
         -0.1941,  0.0185,  0.0895,  0.1059,  0.2352,  0.0381,  0.1332,  0.0310],
        [ 0.0496, -0.1836,  0.1762, -0.0700, -0.1048, -0.1848,  0.1856, -0.0505,
         -0.1311, -0.1946, -0.1530, -0.2420, -0.0441,  0.0900, -0.0885, -0.0771],
        [-0.0822, -0.2342, -0.1330,  0.0716, -0.1040, -0.0387,  0.1673,  0.0490,
         -0.0978, -0.0585,  0.1109,  0.0281,  0.0678,  0.0737, -0.1800, -0.1981],
        [ 0.2351,  0.0743, -0.2024,  0.1219, -0.1569,  0.2129, -0.1537,  0.0374,
         -0.0047, -0.0469, -0.1243, -0.0908,  0.0253, -0.1333,  0.1673,  0.2139],
        [ 0.0063,  0.2349,  0.1194,  0.1672, -0.1216,  0.2160, -0.2494,  0.2291,
          0.1965, -0.0401,  0.1115,  0.0607, -0.1332,  0.0415,  0.0129, -0.1823],


In [23]:
# Access the gradients of the weight of each linear layer
grads0 = weight0.grad
grads1 = weight1.grad
grads2 = weight2.grad

print(grads0)  # FIXME: why getting None?

None


In [24]:
# Update the weights using the learning rate and the gradients
lr = 0.001
# FIXME
# weight0 = weight0 - (lr * grads0)
# weight1 = weight1 - (lr * grads1)
# weight2 = weight2 - (lr * grads2)

In [25]:
import torch.optim as optim

# Create the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()

pred = torch.tensor([[-0.1738,  0.1308]])
target = torch.tensor([[1., 0.]])

loss = criterion(pred, target)
# loss.backward()   # FIXME: RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

# Update the model's parameters using the optimizer
# optimizer.step()

In [26]:
# lecture 7 exercises
y_hat = np.array(10)
y = np.array(1)

# Calculate the MSELoss using NumPy
mse_numpy = np.mean((y_hat - y)**2)

# Create the MSELoss function
criterion = nn.MSELoss()

# Calculate the MSELoss using the created loss function
mse_pytorch = criterion(torch.tensor(y_hat).float(), torch.tensor(y).float())
print(mse_pytorch)

tensor(81.)


In [27]:
model = nn.Sequential(
    nn.Linear(in_features=4, out_features=2, bias=True),
    nn.Sigmoid(),
    nn.Linear(in_features=2, out_features=1, bias=True)
)

criterion = nn.MSELoss()

In [28]:
def show_results(model, dataloader):
    model.eval()
    iter_loader = iter(dataloader)
    for _ in range(3):
        feature, target = next(iter_loader)
        preds = model(feature)
        for p, t in zip(preds, target):
            print(f'Ground truth salary: {t.item():.3f}. Predicted salary: {p.item():.3f}.')

In [29]:
# need to load data for this to work locally
# num_epochs = 10
# # Loop over the number of epochs and the dataloader
# for i in range(num_epochs):
#   for data in dataloader:
#     # Set the gradients to zero
#     optimizer.zero_grad()
#     # Run a forward pass
#     feature, target = data
#     prediction = model(feature)
#     # Calculate the loss
#     loss = criterion(prediction, target)
#     # Compute the gradients
#     loss.backward()
#     # Update the model's parameters
#     optimizer.step()

# show_results(model, dataloader)

### Explanation of the next code cell

First creates a ReLU (Rectified Linear Unit) function using PyTorch's built-in nn.ReLU() function.

Next, it creates a tensor x with a value of -1.0 and sets requires_grad=True. This is important because it tells PyTorch that we want to calculate gradients with respect to x during the backward pass.

Then, it applies the ReLU function to x and stores the result in y.

The y.backward() function is then called to perform a backward pass through the computation graph. This calculates the gradient of y with respect to x.

Finally, the gradient of x is accessed using x.grad and printed out. This is the gradient of the ReLU function at x.

The ReLU function is defined as f(x) = max(0, x). So, its derivative (or gradient) is 1 for x > 0 and 0 for x <= 0. Since x is -1.0 in this case, the gradient at x is 0, which is what the code prints out.

In [36]:
# lecture 8 exercises
# Create a ReLU function with PyTorch
relu_pytorch = nn.ReLU()

# Apply your ReLU function on x, and calculate gradients
x = torch.tensor(-1.0, requires_grad=True)
y = relu_pytorch(x)
y.backward()  # calc's gradient of y wrt x

# Print the gradient of the ReLU function for x
gradient = x.grad  # calculates the gradient at x
print(gradient)

tensor(0.)


In [37]:
# Create a leaky relu function in PyTorch
leaky_relu_pytorch = nn.LeakyReLU(negative_slope = 0.05)

x = torch.tensor(-2.0)
# Call the above function on the tensor x
output = leaky_relu_pytorch(x)
print(output)

tensor(-0.1000)


In [39]:
# Counting the number of parameters
model = nn.Sequential(nn.Linear(16, 4),
                      nn.Linear(4, 2),
                      nn.Linear(2, 1))

total = 0

# Calculate the number of parameters in the model
for parameter in model.parameters():
  total += parameter.numel()

print(total)

81


In [40]:
def calculate_capacity(model):
  total = 0
  for p in model.parameters():
    total += p.numel()
  return total

In [42]:
# Create a neural network with exactly three linear layers and
# less than 120 parameters, which takes n_features as inputs and outputs n_classes.
n_features = 8
n_classes = 2
h1 = 6
h2 = 6

input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

# Create a neural network with less than 120 parameters
model = nn.Sequential(
    nn.Linear(n_features, h1),
    nn.Linear(h1, h2),
    nn.Linear(h2 ,n_classes)
)
output = model(input_tensor)

print(calculate_capacity(model))

110


In [47]:
# Create a neural network with exactly four linear layers and
# more than 120 parameters, which takes n_features as inputs and outputs n_classes.
n_features = 8
n_classes = 2
h = 6
h1 = h
h2 = h
h3 = h

input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

# Create a neural network with more than 120 parameters
model = model = nn.Sequential(
    nn.Linear(n_features, h1),
    nn.Linear(h1, h2),
    nn.Linear(h2, h3),
    nn.Linear(h3 ,n_classes)
)

output = model(input_tensor)

print(calculate_capacity(model))

152
